# 🧩 02. Préparation des Features

Ce notebook prépare les données nettoyées en vue de l’entraînement des modèles de machine learning.

Nous travaillons à partir du fichier CSV nettoyé généré à l’étape précédente : `fiverr_cleaned.csv`.

In [22]:
# 📦 Chargement des bibliothèques nécessaires
import pandas as pd
import numpy as np
import os
import scipy
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

## 📥 1. Chargement du fichier nettoyé

In [23]:
# Charger le DataFrame nettoyé# Chargement du fichier CSV dans un DataFrame pandas

df = pd.read_csv(
    'data/fiverr_gigs/fiverr_cleaned.csv', # Chemin relatif vers le fichier de données
    encoding='UTF-8', # Utilisé pour décoder les caractères spéciaux (utile si UTF-8 génère des erreurs, fréquent avec des accents)
    low_memory=False  # Force pandas à lire tout le fichier avant de deviner les types de colonnes (évite les avertissements ou erreurs)
)

print(f'Données chargées : {df.shape[0]} lignes, {df.shape[1]} colonnes')

# Affiche les 3 premières lignes du DataFrame pour en examiner un aperçu
df.head(3)

Données chargées : 1259 lignes, 6 colonnes


,Description,Lien,Niveau,Prix,Evaluation,Rating_Count
0,"I will do excel formulas, vba macros, charts, ...",https://www.fiverr.com/maltehar/do-excel-formu...,Level 2 Seller,8.701,5.0,56.0
1,"I will do data mining, web scraping, data extr...",https://www.fiverr.com/hotopilams/do-data-mini...,Top Rated Seller,3.263,5.0,1000.0
2,"I will do web scraping , data scraping, scrapi...",https://www.fiverr.com/kodweb/do-web-scraping-...,Level 2 Seller,2.176,5.0,63.0


## 🎯 2. Séparation des variables cibles
Extraction des cibles (y)

In [ ]:
# y_price : colonne 'Prix'
y_prix = df['Prix'].to_numpy()
# y_rating : colonne 'Evaluation'
y_evaluation = df['Evaluation'].to_numpy()
# y_level : colonne 'Niveau'
# Encodez-la numériquement pour la classification :
le = LabelEncoder()
y_niveau = le.fit_transform(df['Niveau'].fillna('Inconnu'))

Vectorisation du texte

In [25]:
tfidf = TfidfVectorizer(
    max_features=2000, 
    stop_words='english'
)
X_text = tfidf.fit_transform(df['Description'].fillna(''))

Encodage du catégoriel

In [26]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_level = ohe.fit_transform(df[['Niveau']].fillna('Inconnu'))

Standardisation uniquement des colonnes numériques

In [27]:
num_cols = ['Prix', 'Evaluation', 'Rating_Count']

X_num = df[num_cols].to_numpy(dtype=float)
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)

Assemblage de la matrice finale

In [ ]:
X = sparse.hstack([
    X_text,
    sparse.csr_matrix(X_level),
    sparse.csr_matrix(X_num_scaled)
], format='csr')

print("Dimentions finale X:", X.shape)

Shape finale X: (1259, 1206)


Sauvegarder dans data/fiverr_features pour rester cohérent avec le reste du projet.

In [29]:
out_dir = 'data/fiverr_features'
os.makedirs(out_dir, exist_ok=True)

scipy.sparse.save_npz(f'{out_dir}/X_features.npz', X)

np.save(f'{out_dir}/y_prix.npy',   y_price)
np.save(f'{out_dir}/y_evaluation.npy',  y_rating)
np.save(f'{out_dir}/y_niveau.npy',   y_level)